In [25]:
!pip install fastai
!pip install aicrowd-cli

In [26]:
from fastai import *
from fastai.vision import *
from fastai.metrics import error_rate, accuracy
import numpy as np
import cv2
import pandas as pd
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import warnings

warnings.filterwarnings('ignore')

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
data_dir = Path("/content/drive/My Drive")
train_labels = pd.read_csv('/content/drive/My Drive/train.csv')
test_ids = pd.read_csv('/content/drive/My Drive/test.csv')

transforms = get_transforms(do_flip = True, flip_vert = True, max_zoom = 1.1, p_affine = 0.75, p_lighting = 0.75, max_warp = 0.2, max_rotate=10.0, max_lighting = 0.1)

test_img = ImageList.from_df(test_ids, path = data_dir/'test_images/test_images', folder = './')

test_img

ImageList (484 items)
Image (3, 464, 464),Image (3, 1080, 1081),Image (3, 890, 891),Image (3, 233, 232),Image (3, 358, 327)
Path: /content/drive/My Drive/test_images/test_images

In [29]:
train_images = (ImageList.from_df(train_labels, path = data_dir/'train_images/train_images', folder='./')
          .split_by_rand_pct(0.01)
          .label_from_df()
          .add_test(test_img)
          .transform(transforms, size = 216)
          .databunch(path='.', bs = 64, device = torch.device('cuda:0')).
          normalize(imagenet_stats)
          )

train_images

ImageDataBunch;

Train: LabelList (9230 items)
x: ImageList
Image (3, 216, 216),Image (3, 216, 216),Image (3, 216, 216),Image (3, 216, 216),Image (3, 216, 216)
y: CategoryList
water,pizza-margherita-baked,broccoli,salad-leaf-salad-green,egg
Path: /content/drive/My Drive/train_images/train_images;

Valid: LabelList (93 items)
x: ImageList
Image (3, 216, 216),Image (3, 216, 216),Image (3, 216, 216),Image (3, 216, 216),Image (3, 216, 216)
y: CategoryList
apple,ham-raw,pizza-margherita-baked,banana,bread-wholemeal
Path: /content/drive/My Drive/train_images/train_images;

Test: LabelList (484 items)
x: ImageList
Image (3, 216, 216),Image (3, 216, 216),Image (3, 216, 216),Image (3, 216, 216),Image (3, 216, 216)
y: EmptyLabelList
,,,,
Path: /content/drive/My Drive/train_images/train_images

In [30]:
lr = 0.0002
learn = cnn_learner(train_images, models.resnet50, metrics = [accuracy])

learn.fit_one_cycle(20, slice(lr))

epoch,train_loss,valid_loss,accuracy,time
0,5.352680,3.904350,0.118280,02:57
1,4.033708,2.713486,0.365591,02:58
2,3.137547,2.231072,0.473118,02:58
3,2.660340,1.904882,0.548387,02:58
4,2.339656,1.776029,0.580645,02:58
5,2.178888,1.768454,0.591398,02:57
6,1.999826,1.623868,0.623656,02:58
7,1.878880,1.661293,0.591398,02:58
8,1.782585,1.599423,0.634409,02:58
9,1.657831,1.582957,0.591398,02:59


In [31]:
labels_list = learn.data.classes

print(labels_list)

['almonds', 'apple', 'avocado', 'banana', 'beer', 'biscuits', 'boisson-au-glucose-50g', 'bread-french-white-flour', 'bread-sourdough', 'bread-white', 'bread-whole-wheat', 'bread-wholemeal', 'broccoli', 'butter', 'carrot', 'cheese', 'chicken', 'chips-french-fries', 'coffee-with-caffeine', 'corn', 'croissant', 'cucumber', 'dark-chocolate', 'egg', 'espresso-with-caffeine', 'french-beans', 'gruyere', 'ham-raw', 'hard-cheese', 'honey', 'jam', 'leaf-spinach', 'mandarine', 'mayonnaise', 'mixed-nuts', 'mixed-salad-chopped-without-sauce', 'mixed-vegetables', 'onion', 'parmesan', 'pasta-spaghetti', 'pickle', 'pizza-margherita-baked', 'potatoes-steamed', 'rice', 'salad-leaf-salad-green', 'salami', 'salmon', 'sauce-savoury', 'soft-cheese', 'strawberries', 'sweet-pepper', 'tea', 'tea-green', 'tomato', 'tomato-sauce', 'water', 'water-mineral', 'white-coffee-with-caffeine', 'wine-red', 'wine-white', 'zucchini']


In [32]:
preds, wts = learn.get_preds(ds_type=DatasetType.Test)

In [33]:
print(preds)

tensor([[1.0329e-06, 1.0029e-07, 2.4721e-08,  ..., 3.6794e-04, 1.6240e-03,
         1.4518e-07],
        [8.4785e-08, 5.6268e-10, 6.9177e-10,  ..., 5.3304e-06, 8.0520e-06,
         3.8287e-08],
        [2.8803e-03, 2.8594e-04, 2.3433e-03,  ..., 3.0397e-05, 2.9857e-03,
         8.8923e-03],
        ...,
        [2.6168e-04, 8.2651e-06, 1.8814e-05,  ..., 1.3028e-04, 5.1850e-04,
         2.7846e-04],
        [1.6100e-02, 1.6616e-03, 4.3526e-03,  ..., 2.8988e-04, 3.4546e-03,
         2.0646e-02],
        [1.5667e-03, 1.1450e-03, 6.3076e-03,  ..., 1.0373e-03, 1.7785e-03,
         1.9099e-04]])


In [34]:
pred_list = []
preds_max = torch.argmax(preds, axis = 1)
print(len(preds_max))

484


In [35]:
for i in range(len(preds_max)):
  pred_list.append(labels_list[preds_max[i]])

print(pred_list)

['water', 'water', 'coffee-with-caffeine', 'hard-cheese', 'bread-wholemeal', 'espresso-with-caffeine', 'almonds', 'bread-wholemeal', 'water', 'espresso-with-caffeine', 'coffee-with-caffeine', 'water', 'banana', 'bread-white', 'coffee-with-caffeine', 'salami', 'banana', 'pasta-spaghetti', 'pasta-spaghetti', 'avocado', 'bread-white', 'chicken', 'carrot', 'bread-wholemeal', 'water', 'water', 'wine-red', 'mandarine', 'pickle', 'water', 'broccoli', 'potatoes-steamed', 'rice', 'water', 'hard-cheese', 'bread-white', 'coffee-with-caffeine', 'water', 'mixed-vegetables', 'pizza-margherita-baked', 'pizza-margherita-baked', 'tea', 'coffee-with-caffeine', 'pizza-margherita-baked', 'espresso-with-caffeine', 'bread-white', 'bread-white', 'pickle', 'salad-leaf-salad-green', 'salmon', 'butter', 'carrot', 'tea', 'wine-white', 'mixed-vegetables', 'tomato', 'rice', 'coffee-with-caffeine', 'salad-leaf-salad-green', 'espresso-with-caffeine', 'salad-leaf-salad-green', 'rice', 'bread-white', 'water', 'salad-l

In [36]:
def copy_csv(filename):
    df = pd.read_csv(filename)
    df.to_csv('submission.csv')

copy_csv('/content/drive/My Drive/test.csv')

In [37]:
df = pd.read_csv('submission.csv')

df["ClassName"] = pred_list

df.to_csv("/content/drive/My Drive/submission3.csv", index = False)

In [38]:
submission = pd.read_csv('/content/drive/My Drive/submission3.csv')

print(submission.head())

   Unnamed: 0         ImageId             ClassName
0           0  90e63a2fde.jpg                 water
1           1  a554d1ca8d.jpg                 water
2           2  48317e8ee8.jpg  coffee-with-caffeine
3           3  79528df667.jpg           hard-cheese
4           4  6d2f2f63f5.jpg       bread-wholemeal


In [39]:
API_KEY = "c3a15f9920d6600174c1d22216f4079f" #Please enter your API Key from [https://www.aicrowd.com/participants/me]
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


In [40]:
!aicrowd submission create -c chunin-exams-food-track-cv-2021 -f '/content/drive/My Drive/submission3.csv'

submission3.csv ━━━━━━━━━━━━━━━━━━━━━ 100.0% • 17.1/15.4 KB • 1.7 MB/s • 0:00:00
                                               ╭─────────────────────────╮                                               
                                               │ Successfully submitted! │                                               
                                               ╰─────────────────────────╯                                               
                                                     Important links                                                     
┌──────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/chunin-exams-food-track-cv-2021/submissions/131189              │
│                  │                                                                                                    │
│  All submissions │ https://www.aicrowd.com/challenges/chunin-ex